#######################################################################################################

Executes daly to update all the stock signals 
Current signals -
1. EMA Cross indicator
2. Turn Around Stocks

#######################################################################################################


In [ ]:
import os, sys

# Get repo root (works both locally & on GitHub Actions)
repo_root = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
src_path = os.path.join(repo_root, "src")

if src_path not in sys.path:
    sys.path.append(src_path)

print("✅ Added to path:", src_path)
print("Current working directory:", os.getcwd())

# Now your imports will work
import get_stocks_data as gs

# from src import __version__
# print(__version__)

✅ Added to path: d:\Work\Projects\Stock_Market\Stock_Market\Stock_signals\Stock-Signals\src
Current working directory: d:\Work\Projects\Stock_Market\Stock_Market\Stock_signals\Stock-Signals\notebooks


ModuleNotFoundError: No module named 'src'

In [2]:
import numpy as np 
import pandas as pd 
import yfinance as yf
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns',120)
import datetime as dt
import datetime
import time 
import ta
import sqlite3

In [ ]:
analysis_period = 30 # past 30 days for Analysis
raw_data_needed = analysis_period + 1 * 365  # +1 year of buffer for calculating indicators
output_period = 30 # restrict output file for EMA-signals in 30 days

In [ ]:
# Determine repository root
# GitHub sets GITHUB_WORKSPACE; locally, we use current working directory
repo_root = os.environ.get("GITHUB_WORKSPACE", os.getcwd())

# if repo_root == os.getcwd(): #then go one directory above to reach repo root
#     repo_root = os.path.abspath(os.path.join(os.getcwd(), os.pardir))

if "d:\\Work" in repo_root or "D:\\Work" in repo_root:
    repo_root = os.path.abspath(os.path.join(os.getcwd(), os.pardir))

print("Repository root determined as:", repo_root)

# Construct full path to the database
db_path = os.path.join(repo_root, "data", "stocks.db")

# Optional: check if it exists
if not os.path.exists(db_path):
    print(f"ERROR: Database not found at {db_path}")
    sys.exit(1)  # stop execution if DB missing

print("Using database at:", db_path)

# Now you can safely call your function
# Example:
# stocklist = gs.get_company_symbols(db_path, 'COMPANY_DETAILS')


Repository root determined as: d:\Work\Projects\Stock_Market\Stock_Market\Stock_signals\Stock-Signals
Using database at: d:\Work\Projects\Stock_Market\Stock_Market\Stock_signals\Stock-Signals\data\stocks.db


ModuleNotFoundError: No module named 'src'

In [ ]:

stocklist = gs.get_company_symbols(db_path, 'COMPANY_DETAILS')
stocklist.head()

tickers = list(stocklist.ticker.unique())
print(len(tickers))
# tickers = ["ESSARSHPNG.NS","LATENTVIEW.NS","BHEL.NS"]

stock_df = gs.fetch_stock_data(tickers, raw_data_needed)

stock_df.head()

2025-11-09 18:32:59,051 - INFO - 📈 Fetching data for 3 tickers...
2025-11-09 18:32:59,056 - INFO - Fetching data from 2023-10-11 to 2025-11-09


d:\Work\Projects\Stock_Market\Stock_Market\Stock_signals\Stock-Signals\data\stocks.db
2178


[*********************100%***********************]  3 of 3 completed
2025-11-09 18:33:00,823 - INFO - ✅ Download completed in 1.77 seconds
2025-11-09 18:33:00,857 - INFO - 📊 Final dataset: 1,537 rows from 2023-10-11 to 2025-11-07


Price,Date,Ticker,Close,High,Low,Open,Volume,20ema,50ema,100ema,200ema,30dv,7dv,Symbol
0,2023-10-11,BHEL.NS,130.466019,132.210883,128.920564,131.413228,18607776.0,130.466019,130.466019,130.466019,130.466019,NaN,NaN,BHEL
1,2023-10-12,BHEL.NS,130.266602,133.357511,129.867781,130.665437,18231128.0,130.447027,130.458198,130.462070,130.464034,NaN,NaN,BHEL
2,2023-10-13,BHEL.NS,127.973366,129.817928,127.125854,129.568661,14077735.0,130.211440,130.360754,130.412789,130.439252,NaN,NaN,BHEL
3,2023-10-16,BHEL.NS,130.765137,131.213828,126.627321,127.724099,15014283.0,130.264173,130.376612,130.419766,130.442494,NaN,NaN,BHEL
4,2023-10-17,BHEL.NS,132.061310,133.207944,129.718215,131.512921,23757135.0,130.435329,130.442679,130.452272,130.458602,NaN,NaN,BHEL


##### Call Accumulation signal

In [6]:
import importlib
import accumulation_signal as acs
importlib.reload(acs)

# Detect accumulation
accumulation_df = acs.detect_gradual_accumulation(stock_df)

# Filter the ones meeting entry criteria
accum_df = acs.filter_accumulation(accumulation_df)


status  = acs.save_accumulation_to_db(accum_df, db_path)
if status:
    print("✅ Database updated successfully")
else:
    print("⚠️ Database Update failed")
# View
accum_df.head()

✅ Database updated successfully


,Symbol,Date,AD_Slope,Above_20EMA_%,Avg_Volume_Spike,RSI,Accumulating,20ema,50ema,200ema,Close,7dv,30dv
0,HUHTAMAKI,2025-10-24,257479.0278,85.0,1.30,70.94,True,247.637933,236.510412,238.301213,265.140015,3.329077e+06,9.714275e+05
1,CENTRALBK,2025-10-24,140672.7339,80.0,1.22,58.09,True,37.930796,37.399357,40.914078,39.070000,1.296222e+07,7.975041e+06
2,COASTCORP,2025-10-24,53964.9821,80.0,1.59,75.57,True,37.591339,36.280382,39.009432,42.840000,9.384134e+05,3.433010e+05
3,DIAMONDYD,2025-10-24,25613.4288,80.0,0.81,58.28,True,1031.158168,1017.889998,1042.609551,1037.500000,1.380521e+05,9.066520e+04


#### EMA Signal indicator - TBC

In [23]:
import importlib
import ema_signal as emasignal
importlib.reload(emasignal)

# Detect Golden/Death cross with volume confirmation

long_stocks, short_stocks, ema_df = emasignal.generate_ema_signals(
    stock_df,
    analysis_period=30,
    vol_thresh=1.5,
    output_period=30
)
long_stocks.head()



status  = emasignal.save_EMA_signal_to_db(long_stocks, db_path)
if status:
    print("✅ Database updated successfully")
else:
    print("⚠️ Database Update failed")
# View
ema_df.head()

1 tickers with Close/200ema crossover between 2025-10-10–2025-11-09
3 tickers with 50ema/200ema crossover between 2025-10-10–2025-11-09
✅ Metadata fetched for 3 stocks
✅ Final signals: 3 long | 0 short
✅ Saved 3 new EMA signal records to SIGNAL_EMA_CROSS
📦 Table now has 3479 total records.
✅ Database updated successfully


,Symbol,signal_1,Date_1,Price_1,7dratio_1,30dratio_1,Vol1,Mom1,signal_2,Date_2,Price_2,7dratio_2,30dratio_2,Vol2,Mom2,currentPrice,Capitalization,MCap,upFrom52wlow,PE,PB,PBXPE,debtToEquity,PS,PricetoCash,EPS,CashperShare,update date,beta,AD,AD_50
0,BHEL,,,0.0,0.00,0.00,0.00,0.0,Golden cross,2025-10-30,261.0,4.52,1.97,4.52,31.0,264.0,Large Cap,92710.0,0.91,164.0,4.0,615.0,0.45,3.17,11.0,2.0,23.0,2025-11-09,0.73,0.87,0.29
1,ESSARSHPNG,,,0.0,0.00,0.00,0.00,0.0,Golden cross,2025-10-31,40.0,0.42,3.01,3.01,29.0,36.0,Micro Cap,829.0,0.65,1.0,-0.0,-0.0,0.00,88.05,117.0,35.0,0.0,2025-11-09,1.12,-0.22,0.27
2,LATENTVIEW,Price_gt_200ema,2025-10-31,448.0,3.03,1.88,3.03,17.0,Golden cross,2025-11-03,475.0,3.14,2.39,3.14,19.0,437.0,Mid Cap,9264.0,0.53,47.0,6.0,263.0,0.02,9.62,16.0,9.0,28.0,2025-11-09,0.30,0.85,0.12


##### Update the Stock returns table

In [35]:
# Update the signal returns 
# import accumulation_signal as acs

import importlib
import signal_returns as sigret
# importlib.reload(sigret)

# from signal_returns import generate_signal_returns

status, final_returns = sigret.generate_signal_returns(db_path, stock_df)

if status:
    print("✅ Signal returns generation successful!")
    display(final_returns.head())
else:
    print("❌ Signal returns generation failed.")


✅ Total signals loaded: 2047
✅ Total signals after merging: 308259
📦 Existing SIGNAL_RETURNS records: 0
✅ New unique records found: 2026
💾 SIGNAL_RETURNS table updated successfully.
🔒 Database connection closed.
✅ Signal returns generation successful!


,Symbol,Signal_Type,Signal_date,Signal_Price,ret_1w,ret_1w_price,ret_1w_date,ret_2w,ret_2w_price,ret_2w_date,ret_1m,ret_1m_price,ret_1m_date,ret_3m,ret_3m_price,ret_3m_date,ret_6m,ret_6m_price,ret_6m_date,ret_1y,ret_1y_price,ret_1y_date,ret_sinceSignal,current_price,current_date,ret_1w_perc,ret_2w_perc,ret_1m_perc,ret_3m_perc,ret_6m_perc,ret_1y_perc,ret_sinceSignal_perc
0,20MICRONS,Golden cross,2025-07-17,254.0,6.43,270.329987,2025-07-24,-7.69,234.470001,2025-07-31,-10.47,227.399994,2025-08-18,-14.98,215.960007,2025-10-15,NaN,NaN,NaT,NaN,NaN,NaT,-15.52,214.570007,2025-10-17,6.43,-7.69,-10.47,-14.98,<NA>,<NA>,-15.52
1,20MICRONS,Golden cross,2025-07-16,259.0,6.86,276.779999,2025-07-23,-5.46,244.850006,2025-07-30,-12.20,227.399994,2025-08-18,-17.18,214.500000,2025-10-14,NaN,NaN,NaT,NaN,NaN,NaT,-17.15,214.570007,2025-10-17,6.86,-5.46,-12.2,-17.18,<NA>,<NA>,-17.15
2,20MICRONS,Golden cross,2025-06-17,237.0,-4.12,227.243713,2025-06-24,-2.58,230.897217,2025-07-01,7.12,253.863495,2025-07-17,-4.95,225.270004,2025-09-15,NaN,NaN,NaT,NaN,NaN,NaT,-9.46,214.570007,2025-10-17,-4.12,-2.58,7.12,-4.95,<NA>,<NA>,-9.46
3,360ONE,Golden cross,2025-06-12,1081.0,3.09,1114.400024,2025-06-19,7.76,1164.849976,2025-06-26,9.92,1188.199951,2025-07-14,-0.28,1078.000000,2025-09-10,NaN,NaN,NaT,NaN,NaN,NaT,7.76,1164.900024,2025-10-17,3.09,7.76,9.92,-0.28,<NA>,<NA>,7.76
4,3MINDIA,Accumulation Signal,2025-08-07,32000.0,-5.16,30350.000000,2025-08-14,-3.52,30875.000000,2025-08-21,-3.64,30835.000000,2025-09-08,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaT,-7.61,29565.000000,2025-10-17,-5.16,-3.52,-3.64,<NA>,<NA>,<NA>,-7.61
